In [1]:
import typing

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import re

from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.metrics.pairwise import cosine_similarity
import config
import src
import requests
import tqdm
import json
import numpy as np
import logging
import torch
device = torch.device('cpu')
from transformers import AutoTokenizer, AutoModel
import cltrier_lib as lib

import yaml

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sstolwi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sstolwi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
 

In [2]:
CFG = config.Config()

In [6]:
dataset: pd.DataFrame = pd.read_csv('data/publicsphere/full_data.csv')
dataset

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,dislikeCount_video,likeCount_video,date_difference,commentCount_video,replyCount_comment,topic,subscribers,HATELIST_FOCUSED_DUMMY,Time_comment_year,Time_video_year
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2017,2017.0
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,195.0,3817.0,743.0,1748.0,NaN,economy,3630000.0,0,2019,2019.0
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019,NaN
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018,2018.0
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3857,8/19/2021 14:50:13,8/19/2021 14:54:28,62.194.51.29,1,6,1152219467579100000,10000695,0,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019,NaN
3858,8/19/2021 15:10:27,8/19/2021 15:12:21,62.194.51.29,1,6,1085362296472430000,10007008,1,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019,NaN
3859,10/6/2021 16:08:39,10/6/2021 16:10:42,213.127.113.113,1,6,UghFY3QJ6nmT_ngCoAEC.7-H0Z7--wxd8goqpaPs-bl,20000102,0,3,1,...,2820.0,12475.0,3803.0,4785.0,NaN,east,6740000.0,0,2018,2010.0
3860,10/15/2021 18:30:04,10/15/2021 18:35:40,213.127.109.191,1,6,UgyWabsmmnq3zam4DgZ4AaABAg,20000418,2,3,1,...,118.0,31761.0,1531.0,2206.0,0.0,east,6800000.0,0,2018,2015.0


In [5]:
#first just try pre-processing and a simple tf-ivf:
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')

In [35]:
def preprocess(texts):
    x_train = []
    for sent in tqdm.tqdm(texts):
        sent = re.sub(r'@[^ ]+', '', sent)  #remove all usernames
        sent = re.sub(r'https?://[^ ]+', '', sent) #remove all hyperlinks
        sent = re.sub(r'#', '', sent) #remove all hashtags
        sent = re.sub(r'([A-Za-z])\1{2,}', r'\1', sent) #normalize language use by replacing duplicate letters by single letters
        sent = re.sub("[^a-zA-Z ]", "", sent) #remove all non-words
        sent = sent.lower().split()
        sent = [lemmatizer.lemmatize(word) for word in sent if word not in set(stop_words)]
        sent = ' '.join(sent)
        x_train.append(sent)
    return x_train

In [36]:
X = preprocess(dataset["commentText"])

100%|██████████| 3862/3862 [00:05<00:00, 654.85it/s] 


In [37]:
tfidf = TfidfVectorizer(max_features=5000, analyzer='word', ngram_range=(1,2), stop_words='english')
X_tfidf = tfidf.fit_transform(X).toarray()

In [12]:
X_tfidf[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [73]:
dataset['tfidf_embedding'] = [torch.tensor(X_tfidf[i], dtype=torch.float32).flatten().tolist() for i in range(X_tfidf.shape[0])]


In [ ]:
dataset['tfidf_embedding']

In [38]:
cosine_similarities = cosine_similarity(X_tfidf)
cosine_similarities

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.0110684 , 0.06654491,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.08009685, 0.        ,
        0.        ],
       ...,
       [0.        , 0.0110684 , 0.08009685, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.06654491, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [54]:
num_documents = len(cosine_similarities)

# Initialize a list to store the most similar document pairs
top_similar_pairs = []

# Iterate through all document pairs
for i in range(num_documents):
    for j in range(i + 1, num_documents):
        similarity = cosine_similarities[i][j]
        
        # Check if the similarity is NaN or zero
        if not np.isnan(similarity) and similarity >= 1:
            pair = (i, j)
            top_similar_pairs.append((similarity, pair))
            
# Sort the list based on similarity in descending order
top_similar_pairs.sort(key=lambda x: x[0], reverse=True)
            


In [56]:
top_similar_pairs

[(1.0000000000000002, (139, 1990)),
 (1.0000000000000002, (246, 1179)),
 (1.0000000000000002, (362, 736)),
 (1.0000000000000002, (722, 739)),
 (1.0, (21, 1856)),
 (1.0, (75, 366)),
 (1.0, (75, 1022)),
 (1.0, (75, 2122)),
 (1.0, (75, 2421)),
 (1.0, (75, 2946)),
 (1.0, (101, 850)),
 (1.0, (133, 1057)),
 (1.0, (141, 3838)),
 (1.0, (164, 1240)),
 (1.0, (164, 2182)),
 (1.0, (198, 3213)),
 (1.0, (242, 2779)),
 (1.0, (242, 3278)),
 (1.0, (259, 490)),
 (1.0, (313, 2607)),
 (1.0, (341, 352)),
 (1.0, (366, 1022)),
 (1.0, (366, 2122)),
 (1.0, (366, 2421)),
 (1.0, (366, 2946)),
 (1.0, (395, 3340)),
 (1.0, (400, 585)),
 (1.0, (400, 877)),
 (1.0, (400, 962)),
 (1.0, (400, 2099)),
 (1.0, (400, 2848)),
 (1.0, (400, 3752)),
 (1.0, (436, 3211)),
 (1.0, (487, 936)),
 (1.0, (487, 2484)),
 (1.0, (546, 911)),
 (1.0, (559, 3708)),
 (1.0, (585, 877)),
 (1.0, (585, 962)),
 (1.0, (585, 2099)),
 (1.0, (585, 2848)),
 (1.0, (585, 3752)),
 (1.0, (643, 1401)),
 (1.0, (673, 1743)),
 (1.0, (678, 2244)),
 (1.0, (678, 2

In [55]:
result = [pair for value, pair in top_similar_pairs if value >= 1]
print(result)

[(139, 1990), (246, 1179), (362, 736), (722, 739), (21, 1856), (75, 366), (75, 1022), (75, 2122), (75, 2421), (75, 2946), (101, 850), (133, 1057), (141, 3838), (164, 1240), (164, 2182), (198, 3213), (242, 2779), (242, 3278), (259, 490), (313, 2607), (341, 352), (366, 1022), (366, 2122), (366, 2421), (366, 2946), (395, 3340), (400, 585), (400, 877), (400, 962), (400, 2099), (400, 2848), (400, 3752), (436, 3211), (487, 936), (487, 2484), (546, 911), (559, 3708), (585, 877), (585, 962), (585, 2099), (585, 2848), (585, 3752), (643, 1401), (673, 1743), (678, 2244), (678, 2481), (678, 2843), (678, 2856), (678, 2971), (702, 1392), (709, 1800), (753, 930), (753, 1564), (753, 1887), (753, 2728), (793, 2935), (813, 1872), (816, 1088), (877, 962), (877, 2099), (877, 2848), (877, 3752), (894, 1060), (914, 2707), (914, 2914), (919, 3578), (930, 1564), (930, 1887), (930, 2728), (936, 2484), (937, 1005), (937, 1192), (955, 1404), (962, 2099), (962, 2848), (962, 3752), (969, 1196), (969, 1449), (969, 

In [62]:
for first, second in result:
    print(dataset["commentText"][[first, second]])
    print()

139          vote trump
1990    Vote Trump 2020
Name: commentText, dtype: object

246                        hell yeah fam
1179    @TheDailyShow Hells to the Yeah.
Name: commentText, dtype: object

362                @James Persinger \n apple and orange.
736    @hardball @MSNBC Just a few apples and oranges...
Name: commentText, dtype: object

722             @hardball @KenSalazar Good grief!
739    @NBCNews My opinion......lol.  Good grief.
Name: commentText, dtype: object

21                     Mika is low IQ
1856    @Rusty you still very low IQ.
Name: commentText, dtype: object

75                                               I agree
366    Kai Watson So you agree that Haiti is a Shithole?
Name: commentText, dtype: object

75                                 I agree
1022    @RealTimers @SteveSchmidtSES Agree
Name: commentText, dtype: object

75                 I agree
2122    Rtb boone i agree.
Name: commentText, dtype: object

75                  I agree
2421    Braulio V.  I agre

In [61]:
print(X[246])
print(X[1179])
print(X[139])
print(X[1990])
print(X[21])
print(X[1856])
print(X[722])
print(X[739])
print(X[362])
print(X[736])

hell yeah fam
hell yeah
vote trump
vote trump
mika low iq
still low iq
good grief
opinionlol good grief
persinger apple orange
apple orange


In [74]:
grouped_data = dataset.groupby("Platform")
dist = torch.nn.PairwiseDistance()
resultsplatform: typing.Dict[typing.Tuple[str, str], float] = {}
for model_1, c_1 in tqdm.tqdm(grouped_data['tfidf_embedding'], total=grouped_data.ngroups):
    for model_2, c_2 in tqdm.tqdm(grouped_data['tfidf_embedding'], total=grouped_data.ngroups):

        if (
            (model_1, model_2) in resultsplatform.keys() or 
            (model_2, model_1) in resultsplatform.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        resultsplatform[(model_1, model_2)] = res

        print(f'{model_1}:{model_2}:{res.item()}')

KeyboardInterrupt: 

In [ ]:
grouped_data = dataset.groupby("topiccode")
dist = torch.nn.PairwiseDistance()
resultstopic: typing.Dict[typing.Tuple[str, str], float] = {}
for model_1, c_1 in tqdm.tqdm(grouped_data['tfidf_embedding'], total=grouped_data.ngroups):
    for model_2, c_2 in tqdm.tqdm(grouped_data['tfidf_embedding'], total=grouped_data.ngroups):

        if (
            (model_1, model_2) in resultstopic.keys() or 
            (model_2, model_1) in resultstopic.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        resultstopic[(model_1, model_2)] = res

        print(f'{model_1}:{model_2}:{res.item()}')

In [ ]:
grouped_data = dataset.groupby("Genre")
dist = torch.nn.PairwiseDistance()
resultsgenre: typing.Dict[typing.Tuple[str, str], float] = {}
for model_1, c_1 in tqdm.tqdm(grouped_data['tfidf_embedding'], total=grouped_data.ngroups):
    for model_2, c_2 in tqdm.tqdm(grouped_data['tfidf_embedding'], total=grouped_data.ngroups):

        if (
            (model_1, model_2) in resultsgenre.keys() or 
            (model_2, model_1) in resultsgenre.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        resultsgenre[(model_1, model_2)] = res

        print(f'{model_1}:{model_2}:{res.item()}')

In [4]:
MODEL: str = 'mixtral:8x7b-instruct-v0.1-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'


In [5]:
embed_MXBAI: typing.Dict[str, np.ndarray] = {}

In [10]:
context = 'social media replies to a news- or infotainment-post'
for index, row in tqdm.tqdm(dataset["commentText"].items(), total=len(dataset)):
    try: 
        embed = np.array(requests.post(
            'https://inf.cl.uni-trier.de/embed/',
            json={'model': MODEL, 
                  'prompt': 'You help me get embeddings for a sentence. I provide you with a context and a sentence and you reply only with that exact sentence. Context = ' + context + '; Sentence: ' + row}
            ).json()["response"])
    except Exception as _e:
        logging.warning(_e)
        embed = None
    
    embed_MXBAI[index] = embed

  3%|▎         | 123/3862 [04:58<2:31:24,  2.43s/it]


KeyboardInterrupt: 

In [ ]:
dataset_w_embeds = dataset.join(pd.Series(embed_MXBAI, name="embed_MXBAI"))
dataset_w_embeds.to_parquet(f'{CFG.report_dir}/pubsphere.MXBAIembeds.parquet')
dataset_w_embeds.head()

In [3]:
#load datasets:
dataset_w_embeds = pd.read_parquet(f'{CFG.report_dir}/pubsphere.MXBAIembeds.parquet')
dataset_prompt = pd.read_parquet(f'{CFG.report_dir}\publicsphere.cardiff_prompt_classify_s.parquet')


<>:3: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\p'
C:\Users\sstolwi\AppData\Local\Temp\ipykernel_13408\795071030.py:3: SyntaxWarning: invalid escape sequence '\p'
  dataset_prompt = pd.read_parquet(f'{CFG.report_dir}\publicsphere.cardiff_prompt_classify_s.parquet')


In [ ]:
#create BERT-base embeddings for each word:

model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [ ]:
#create Twhin-BERT-base embeddings for each word:
model_name = "Twitter/twhin-bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [12]:
#calculate DiffCSE embeddings:
model_name = "voidism/diffcse-roberta-base-sts"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/Users/SjoerdStolwijk/anaconda3/envs/DefiningDebateQuality/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at voidism/diffcse-roberta-base-sts were not used when initializing RobertaModel: ['aux_bert.embeddings.LayerNorm.bias', 'aux_bert.embeddings.LayerNorm.weight', 'aux_bert.embeddings.position_embeddings.weight', 'aux_bert.embeddings.position_ids', 'aux_bert.embeddings.token_type_embeddings.weight', 'aux_bert.embeddings.word_embeddings.weight', 'aux_bert.encoder.layer.0.attention.output.LayerNorm.bias', 'aux_bert.encoder.layer.0.attention.output.LayerNorm.weight', 'aux_bert.encoder.layer.0.attention.output.dense.bias', 'aux_bert.encoder.layer.0.attention.output.dense.weight', 'aux_bert.encoder.layer.0.attention.self.key.bias'

In [13]:
#function for encoding sentence embeddings based on the DiffCSE approach which is based on (and refers to) SimCSE/evaluation.py 
def encode_sentence(sentence):
    input_ids = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, max_length=512, padding='max_length')
    input_ids_tensor = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension
    attention_mask = torch.ones_like(input_ids_tensor)  # Creating attention mask 
    with torch.no_grad():
            outputs = model(input_ids_tensor, output_hidden_states=True, return_dict=True, attention_mask=attention_mask)
            sentence_embedding = outputs.last_hidden_state[:, 0].cpu()
            return sentence_embedding.tolist()

In [ ]:
#for DiffCSE: Create embeddings for each argument and store them in a new column
dataset.loc[:, 'DiffCSE_embedding'] = dataset["commentText"].apply(encode_sentence)

In [7]:
#claim mine the publicsphere data
#first add system prompt


In [4]:
SYSTEM_claim: str = \
    """
        Instruction:

        You are a text annotation assitant. Analyze a collection of social media comments enclosed in chevrons <..>. Identify and list the claims within these comments. Claims can be related to events, issues, opinions or concerns in relation to the specified topic.
        Claims are defined as the main assertion or conclusion of an argument.
        You summarize each claim into a short simple sentence.

        Response format:

        You provide only the list of claims, separated by commas, without any additional text or explanations. If no claims can be identified, return an empty list [].

        Response format template:
        
        ["claim 1", "claim 2", ... "claim x"]
	"""

In [5]:
MODEL: str = 'llama3.1:70b-instruct-q6_K'  # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'


In [6]:
dataset_w_embeds.columns

Index(['StartDate', 'RecordedDate', 'IPAddress', 'Finished', 'Coder', 'ID',
       'Mark_ID', 'Genre', 'topiccode', 'Platform', 'Anonymity',
       'Anonymity_9_TEXT', 'codable', 'Interaction', 'Acknowledgement',
       'TopicRelevance', 'Reasoning', 'BackgroundInfo', 'ExternalEvidence',
       'ExternalEvidence_1_TEXT', 'Opinion', 'disagreement',
       'Ideologicaldirection', 'Name_calling', 'Vulgarity',
       'Attack_reputation', 'Question_Intelligenc', 'All_caps_function',
       'Sarcasm_to_criticize', 'Individual_right', 'discrimination',
       'Invoke_violence', 'Tone', 'INTERACTIVITY_DUMMY', 'RATIONALITY_DUMMY',
       'HAS_OPINION_DUMMY', 'LIBERAL_NEUTRAL_CONSERVATIVE', 'LIBERAL_DUMMY',
       'CONSERVATIVE_DUMMY', 'NAMECALLING_DUMMY', 'VULGAR_DUMMY',
       'NAMECALLING_VULGAR_DUMMY', 'INCIVILITY_ORDINAL', 'INCIVILITY_DUMMY',
       'INTOLERANCE_DUMMY', 'filter_$', 'IMPOLITENESS_DUMMY', 'commentText',
       'showName', 'genre', 'Time_comment', 'likeCount_comment', 'entitie

topiccode staat voor:
0 : YouTube general
1 : YT Mueller/Comy investigation
2 : YT: Economy
3 : YT: Middle East
4 : Twitter:general

In [24]:
topicdict = {0 : 'YTgen', 
1 : 'YT_investigation',
2 : 'YT_Econ',
3 : 'YT_ME',
4 : 'Twitgen'}

dataset_w_embeds.loc[:,'topiccodeSTR']= dataset_w_embeds.topiccode.map(topicdict)

In [26]:
dataset_prompt.loc[:,'topiccodeSTR']= dataset_prompt.topiccode.map(topicdict)

In [51]:
options_str = """
seed: 42
temperature: 0.8
num_predict: 1
"""

options = yaml.safe_load(options_str)

In [51]:

requests.post(
                            'https://inf.cl.uni-trier.de/',
                            json={
                                'model': MODEL,
                                'system': SYSTEM_claim,
                                'prompt': f'The following set of social media posts are replies to a news- or infotainment-post. '
                                        + f'Check whether your answer only consists of a list of claims. \n"Posts":\n<{dataset_w_embeds["commentText"][:5].to_list()}>',
                                'options': options
                                }).json()

{'id': '196415a8-7027-11ef-8a8f-0d8e6fdd3d4b',
 'timestamp': '2024-09-11T10:17:50.380071',
 'model': 'llama3.1:70b-instruct-q6_K',
 'prompt': [{'role': 'system',
   'content': '\n        Instruction:\n\n        You are a text annotation assitant. Analyze a collection of social media comments enclosed in chevrons <..>. Identify and list the claims within these comments. Claims can be related to events, issues, opinions or concerns in relation to the specified topic.\n        Claims are defined as the main assertion or conclusion of an argument.\n        You summarize each claim into a short simple sentence.\n\n        Response format:\n\n        You provide only the list of claims, separated by commas, without any additional text or explanations. If no claims can be identified, return an empty list [].\n\n        Response format template:\n        \n        ["claim 1", "claim 2", ... "claim x"]\n\t'},
  {'role': 'user',
   'content': 'The following set of social media posts are social m

In [52]:
#get a list of claims per comment:
chunked_result: typing.List[pd.DataFrame] = []
for index, row in tqdm.tqdm(dataset_prompt.iterrows()):
    try: 
        chunked_result.append(
            pd.DataFrame(
                data=[
                    requests.post(
                        'https://inf.cl.uni-trier.de/',
                        json={
                            'model': MODEL,
                            'system': SYSTEM_claim,
                            'prompt': f'The following set of social media posts are replies to a news- or infotainment-post. '
                                    + f'Check whether your answer strictly adheres to the specified format. \n"Posts":\n<{row["commentText"]}>',
                            'options': options
                            }).json()['response']                       
                ],
                columns=['claims']
            )
        )
        
    except json.JSONDecodeError:
        print("invalid json response, skipping to next batch")

3862it [2:05:28,  1.95s/it]


In [27]:
#apply generalized claim mining to get a list of claims:
chunked_result: typing.List[pd.DataFrame] = []
for label, group in dataset_prompt[:5].groupby('topiccodeSTR'):
    for index, row in tqdm.tqdm(group.iterrows()):
        try: 
            chunked_result.append(
                pd.DataFrame(
                    data=[
                        requests.post(
                            'https://inf.cl.uni-trier.de/',
                            json={
                                'model': MODEL,
                                'system': SYSTEM_claim,
                                'prompt': f'The following set of social media posts are replies to a news- or infotainment-post. '
                                        + f'Check whether your answer strictly adheres to the specified format. \n"Posts":\n<{row["commentText"]}>'
                                }).json()['response']                       
                    ],
                    columns=['claims']
                )
                .assign(label=label)
            )
            
        except json.JSONDecodeError:
            print("invalid json response, skipping to next batch")

0it [00:00, ?it/s]

1it [00:16, 16.55s/it]
1it [00:02,  2.04s/it]
3it [00:04,  1.40s/it]


In [54]:
claim_df = pd.concat(chunked_result, ignore_index=True)
print(claim_df)

                                                 claims
0                                                    []
1     ["Minerals have rights too", "A box of rocks w...
2     ["There is a question about whether Trump was ...
3     ["The death of free and civil dialogue is evid...
4                         ["Nobody else will hug him."]
...                                                 ...
3857  ["They can't afford chemical peels and facelif...
3858                           ["He will be confirmed"]
3859  ["They knew all about the cameras.", "The susp...
3860  ["Troops are waiting in the fields", "Men will...
3861  ["Russia and the US are both against ISIS", "T...

[3862 rows x 1 columns]


In [63]:
claim_df.claims.to_list()

['[]',
 '["Minerals have rights too", "A box of rocks was slandered"]',
 '["There is a question about whether Trump was compromised", "It is unknown whether Trump was aware of being compromised"]',
 '["The death of free and civil dialogue is evident in online comments."]',
 '["Nobody else will hug him."]',
 '["The user will never be president", "The user is the daughter of Liawatha"]',
 '["Trump is a traitor"]',
 '["he would shoot someone on Fifth Avenue"]',
 '["America is a country of idiots run by the NRA", "Guns are more valued than children\'s lives in America", "The 2nd amendment should be abolished", "Getting rid of guns will save children\'s lives"]',
 '["They weren\'t going to vote democratic anyway."]',
 '["Ivanka has high moral standards"]',
 '["Hillary is a lesbian"]',
 '[]',
 '["Admitting Russia back in is sensible", "Democrats have gone full neocon for the last three years"]',
 '["You will lose the midterms"]',
 '["A constitutional crisis is occurring"]',
 '["DTJ thinks he

In [49]:
dataset_prompt.claim_run1[:960].tail()

955                                                   []
956    ["Democrats support Dan in 2020", "Someone nam...
957    ["There is a recording that has not been relea...
958                                  ["He had dementia"]
959    ["Only Red States are prone to natural disaste...
Name: claim_run1, dtype: object

In [58]:
#change options to low temperature (0,1) and single prediction and compare result:
options_low = """
seed: 42
temperature: 0.1
num_predict: 1
"""

options_low = yaml.safe_load(options_low)

In [59]:
#test low temperature claims per comment:
chunked_result: typing.List[pd.DataFrame] = []
for index, row in tqdm.tqdm(dataset_prompt[:5].iterrows()):
    try: 
        chunked_result.append(
            pd.DataFrame(
                data=[
                    requests.post(
                        'https://inf.cl.uni-trier.de/',
                        json={
                            'model': MODEL,
                            'system': SYSTEM_claim,
                            'prompt': f'The following set of social media posts are replies to a news- or infotainment-post. '
                                    + f'Check whether your answer strictly adheres to the specified format. \n"Posts":\n<{row["commentText"]}>',
                            'options': options
                            }).json()['response']                       
                ],
                columns=['claims']
            )
        )
        
    except json.JSONDecodeError:
        print("invalid json response, skipping to next batch")

5it [00:05,  1.18s/it]


In [61]:
claim_low = pd.concat(chunked_result, ignore_index=True)
print(claim_low, claim_df)

                                              claims
0                                                 []
1  ["Minerals have rights too", "A box of rocks w...
2                ["Trump may have been compromised"]
3  ["The death of free and civil dialogue is evid...
4                      ["Nobody else will hug him."]                                                  claims
0                                                    []
1     ["Minerals have rights too", "A box of rocks w...
2     ["There is a question about whether Trump was ...
3     ["The death of free and civil dialogue is evid...
4                         ["Nobody else will hug him."]
...                                                 ...
3857  ["They can't afford chemical peels and facelif...
3858                           ["He will be confirmed"]
3859  ["They knew all about the cameras.", "The susp...
3860  ["Troops are waiting in the fields", "Men will...
3861  ["Russia and the US are both against ISIS", "T...

[3862 row

In [53]:
#list the claims in a way that links them to the original post, so we can group the claims according to any grouper, while still being able to find unique embeddings per claim: 
claim_df['original_index'] = claim_df.index
claim_df['claims'] = claim_df['claims'].str.split('",')
claim_df = claim_df.explode('claims')
claim_df = claim_df.reset_index(drop=True)
claim_df = claim_df.drop_duplicates()
print(claim_df)

                                                 claims  original_index
0                                                    []               0
1                                ["Minerals have rights               1
2      "The post is a vicious insult to boxes of roc...               1
3                     ["Trump may have been compromised               2
4      "The extent of Trump's awareness about his ow...               2
...                                                 ...             ...
1408                    ["Democrats support Dan in 2020             956
1409          "Someone named Mica should be destroyed"]             956
1410  ["There is a recording that has not been relea...             957
1411                                ["He had dementia"]             958
1412  ["Only Red States are prone to natural disaste...             959

[1413 rows x 2 columns]


In [56]:
dataset_prompt.loc[:, "claim_optdef"]=claim_df

In [57]:
dataset_prompt.to_parquet(f'{CFG.report_dir}/pubsphere.claim.parquet')

In [15]:
#load data:
dataset_prompt = pd.read_parquet(f'{CFG.report_dir}/pubsphere.claim.parquet')

In [21]:
dataset_prompt.head()

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,sentiment,offensive,topics,emotions,irony,hate,topiccodeSTR,claim_run1,claim_optdef,claim_optdef_embed_MXBAI
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,"b'[""negative"",""neutral""]'","{'offensive', 'non-offensive'}",set(),"{'sadness', 'pessimism'}","{'non_irony', 'irony'}",{'NOT-HATE'},YTgen,[],[],"[-0.5064548254013062, -0.11242958903312683, 0...."
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,"b'[""negative"",""neutral""]'","{'offensive', 'non-offensive'}",{'news_&_social_concern'},"{'disgust', 'anger'}",{'non_irony'},{'NOT-HATE'},YT_Econ,"[""Minerals have rights"", ""The post is a viciou...","[""Minerals have rights too"", ""A box of rocks w...","[0.07165694236755371, 0.056573204696178436, 0...."
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,"b'[""negative"",""neutral""]'",{'non-offensive'},{'news_&_social_concern'},"{'anticipation', 'disgust', 'anger'}",{'non_irony'},{'NOT-HATE'},Twitgen,"[""Trump may have been compromised"", ""The exten...","[""There is a question about whether Trump was ...","[0.7152484059333801, -0.363998144865036, -0.46..."
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,"b'[""negative"",""neutral""]'",{'non-offensive'},{'news_&_social_concern'},{'disgust'},{'irony'},{'NOT-HATE'},YTgen,"[""There are only a few people left who value c...","[""The death of free and civil dialogue is evid...","[-0.19108964502811432, -0.04326176643371582, -..."
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,"b'[""positive"",""neutral""]'",{'non-offensive'},{'diaries_&_daily_life'},{'sadness'},"{'non_irony', 'irony'}",{'NOT-HATE'},YTgen,"[""Nobody wants to hug him""]","[""Nobody else will hug him.""]","[0.4981515407562256, -0.1532106101512909, 0.31..."


In [33]:
#get MXBAI embeddings for the claims, but ordered per post, so one embedding for the set of claims for each post:
context = 'claims made in social media replies to a news- or infotainment-post'

claim_optdef_embed_MXBAI: typing.Dict[str, np.ndarray] = (
    pd.Series(src.SentenceEmbedder()(dataset_prompt["claim_optdef"], prefix='You help me get embeddings for a sentence. I provide you with a context and a sentence and you reply only with that exact sentence. Context = ' + context + '; Sentence: ' ), name="claim_optdef_embed_MXBAI")
    .rename_axis(index=dataset_prompt.index.names)
)
claim_optdef_embed_MXBAI

100%|██████████| 3862/3862 [14:04<00:00,  4.57it/s]


0       [0.3085225224494934, 0.0025453902781009674, -0...
1       [0.3428829610347748, 0.03502892702817917, -0.5...
2       [0.5012139081954956, -0.1079118549823761, -0.6...
3       [0.21832342445850372, -0.13364076614379883, -0...
4       [0.49281346797943115, -0.11275702714920044, -0...
                              ...                        
3857    [0.5127694606781006, -0.30040407180786133, -0....
3858    [0.5046452283859253, -0.13685280084609985, -0....
3859    [0.5013481378555298, -0.34943655133247375, -0....
3860    [0.4138392210006714, -0.20430096983909607, -0....
3861    [0.3057856261730194, -0.03770238161087036, -0....
Name: claim_optdef_embed_MXBAI, Length: 3862, dtype: object

In [35]:
#dataset_prompt.loc[:,'claim_optdef_embed_MXBAI'] = claim_optdef_embed_MXBAI
dataset_prompt.to_parquet(f'{CFG.report_dir}/pubsphere.claim_embed.parquet')
#dataset_prompt.head()

In [17]:
claim_optdef_embed_MXBAI

0       [-0.5064548254013062, -0.11242958903312683, 0....
1       [0.07165694236755371, 0.056573204696178436, 0....
2       [0.7152484059333801, -0.363998144865036, -0.46...
3       [-0.19108964502811432, -0.04326176643371582, -...
4       [0.4981515407562256, -0.1532106101512909, 0.31...
                              ...                        
3857    [0.5195960402488708, -0.3683280944824219, -0.4...
3858    [0.24329741299152374, -0.7627171874046326, 0.2...
3859    [0.6211866736412048, -0.27806738018989563, -0....
3860    [-0.26269903779029846, -0.3088279068470001, 0....
3861    [-0.16621819138526917, -0.10346470773220062, -...
Name: claim_optdef_embed_MXBAI, Length: 3862, dtype: object

In [22]:
dataset_prompt = dataset_prompt.join(claim_optdef_embed_MXBAI)
dataset_prompt.to_parquet(f'{CFG.report_dir}/pubsphere.claim_embed.parquet')
dataset_prompt.head()

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,sentiment,offensive,topics,emotions,irony,hate,topiccodeSTR,claim_run1,claim_optdef,claim_optdef_embed_MXBAI
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,"b'[""negative"",""neutral""]'","{'offensive', 'non-offensive'}",set(),"{'sadness', 'pessimism'}","{'non_irony', 'irony'}",{'NOT-HATE'},YTgen,[],[],"[-0.5064548254013062, -0.11242958903312683, 0...."
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,"b'[""negative"",""neutral""]'","{'offensive', 'non-offensive'}",{'news_&_social_concern'},"{'disgust', 'anger'}",{'non_irony'},{'NOT-HATE'},YT_Econ,"[""Minerals have rights"", ""The post is a viciou...","[""Minerals have rights too"", ""A box of rocks w...","[0.07165694236755371, 0.056573204696178436, 0...."
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,"b'[""negative"",""neutral""]'",{'non-offensive'},{'news_&_social_concern'},"{'anticipation', 'disgust', 'anger'}",{'non_irony'},{'NOT-HATE'},Twitgen,"[""Trump may have been compromised"", ""The exten...","[""There is a question about whether Trump was ...","[0.7152484059333801, -0.363998144865036, -0.46..."
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,"b'[""negative"",""neutral""]'",{'non-offensive'},{'news_&_social_concern'},{'disgust'},{'irony'},{'NOT-HATE'},YTgen,"[""There are only a few people left who value c...","[""The death of free and civil dialogue is evid...","[-0.19108964502811432, -0.04326176643371582, -..."
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,"b'[""positive"",""neutral""]'",{'non-offensive'},{'diaries_&_daily_life'},{'sadness'},"{'non_irony', 'irony'}",{'NOT-HATE'},YTgen,"[""Nobody wants to hug him""]","[""Nobody else will hug him.""]","[0.4981515407562256, -0.1532106101512909, 0.31..."


In [36]:
#load data:
dataset_prompt = pd.read_parquet(f'{CFG.report_dir}/pubsphere.claim_embed.parquet')
dataset_prompt.head()

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,irony,hate,topiccodeSTR,claim_run1,claim_optdef,claim_optdef_embed_MXBAI,claim_optlow,claim_optlow_MXBAI,tfidf_embed_post,embed_MXBAI_post
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,"{'non_irony', 'irony'}",{'NOT-HATE'},YTgen,[],[],"[0.3085225224494934, 0.0025453902781009674, -0...",[],"[0.3085225224494934, 0.0025453902781009674, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.40566757321357727, -0.032418690621852875, -..."
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,{'non_irony'},{'NOT-HATE'},YT_Econ,"[""Minerals have rights"", ""The post is a viciou...","[""Minerals have rights too"", ""A box of rocks w...","[0.3428829610347748, 0.03502892702817917, -0.5...","[""Minerals have rights too.""]","[0.34888750314712524, -0.01314709335565567, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.49942252039909363, -0.22240903973579407, -0..."
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,{'non_irony'},{'NOT-HATE'},Twitgen,"[""Trump may have been compromised"", ""The exten...","[""There is a question about whether Trump was ...","[0.5012139081954956, -0.1079118549823761, -0.6...","[""The question has always been if Trump was co...","[0.4851573407649994, -0.06400042772293091, -0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.6132397651672363, -0.3024018704891205, -0.6..."
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,{'irony'},{'NOT-HATE'},YTgen,"[""There are only a few people left who value c...","[""The death of free and civil dialogue is evid...","[0.21832342445850372, -0.13364076614379883, -0...","[""There are only a few people left who value f...","[0.31287914514541626, -0.15600241720676422, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.27884048223495483, -0.3008716404438019, -0...."
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,"{'non_irony', 'irony'}",{'NOT-HATE'},YTgen,"[""Nobody wants to hug him""]","[""Nobody else will hug him.""]","[0.49281346797943115, -0.11275702714920044, -0...","[""No one will hug him""]","[0.5331375598907471, -0.04385572671890259, -0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5210463404655457, 0.03200243413448334, -0.3..."


In [37]:
#do we find differences between platforms, topics and genres in the claims made, based on the embeddings of the claims?
grouped_data = dataset_prompt.groupby("Platform")
dist = torch.nn.PairwiseDistance()
resultsplatform: typing.Dict[typing.Tuple[str, str], float] = {}
for model_1, c_1 in tqdm.tqdm(grouped_data['claim_optdef_embed_MXBAI'], total=grouped_data.ngroups):
    for model_2, c_2 in tqdm.tqdm(grouped_data['claim_optdef_embed_MXBAI'], total=grouped_data.ngroups):

        if (
            (model_1, model_2) in resultsplatform.keys() or 
            (model_2, model_1) in resultsplatform.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        resultsplatform[(model_1, model_2)] = res

        print(f'{model_1}:{model_2}:{res.item()}')

  0%|          | 0/2 [02:10<?, ?it/s]


KeyboardInterrupt: 

In [24]:
grouped_data = dataset_prompt.groupby("topiccode")
dist = torch.nn.PairwiseDistance()
resultstopic: typing.Dict[typing.Tuple[str, str], float] = {}
for model_1, c_1 in tqdm.tqdm(grouped_data['claim_optdef_embed_MXBAI'], total=grouped_data.ngroups):
    for model_2, c_2 in tqdm.tqdm(grouped_data['claim_optdef_embed_MXBAI'], total=grouped_data.ngroups):

        if (
            (model_1, model_2) in resultstopic.keys() or 
            (model_2, model_1) in resultstopic.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        resultstopic[(model_1, model_2)] = res

        print(f'{model_1}:{model_2}:{res.item()}')

  0%|          | 0/5 [00:00<?, ?it/s]

0:0:17.067672729492188


0:1:17.221986770629883


0:2:17.286460876464844


0:3:17.40349006652832


...

3:3:17.374549865722656


 80%|████████  | 4/5 [38:48<08:43, 523.38s/it]

3:4:17.5199031829834


100%|██████████| 5/5 [41:56<00:00, 503.27s/it]

4:4:17.18415069580078


In [ ]:
grouped_data = dataset_prompt.groupby("Genre")
dist = torch.nn.PairwiseDistance()
resultstopic: typing.Dict[typing.Tuple[str, str], float] = {}
for model_1, c_1 in tqdm.tqdm(grouped_data['claim_optdef_embed_MXBAI'], total=grouped_data.ngroups):
    for model_2, c_2 in tqdm.tqdm(grouped_data['claim_optdef_embed_MXBAI'], total=grouped_data.ngroups):

        if (
            (model_1, model_2) in resultstopic.keys() or 
            (model_2, model_1) in resultstopic.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        resultstopic[(model_1, model_2)] = res

        print(f'{model_1}:{model_2}:{res.item()}')

In [ ]:
#does the result differ if we calculate embeddings per claim instead of all claims per post?

In [ ]:
#does it matter if we add context to the embedding generation.. we should to remain identical to the post MXBAI embedding generation process...